<a href="https://www.kaggle.com/code/kmljts/pyspark-practice?scriptVersionId=258157485" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr
import pandas as pd
from datetime import datetime, timedelta

In [2]:
def curr_time() -> str:
    return (datetime.now() + timedelta(hours=5, minutes=30)).strftime('%Y-%b-%d_%H-%M-%S').upper()

curr_time()

'2025-AUG-26_01-21-47'

## Opening CSV file in Pandas

In [3]:
pd_df = pd.read_csv('/kaggle/input/employee-dataset/Employee.csv')
pd_df

,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain,LeaveOrNot
0,Bachelors,2017,Bangalore,3,34,Male,No,0,0
1,Bachelors,2013,Pune,1,28,Female,No,3,1
2,Bachelors,2014,New Delhi,3,38,Female,No,2,0
3,Masters,2016,Bangalore,3,27,Male,No,5,1
4,Masters,2017,Pune,3,24,Male,Yes,2,1
...,...,...,...,...,...,...,...,...,...
4648,Bachelors,2013,Bangalore,3,26,Female,No,4,0
4649,Masters,2013,Pune,2,37,Male,No,2,1
4650,Masters,2018,New Delhi,3,27,Male,No,5,1
4651,Bachelors,2012,Bangalore,3,30,Male,Yes,2,0


In [4]:
pd_df.dtypes

Education                    object
JoiningYear                   int64
City                         object
PaymentTier                   int64
Age                           int64
Gender                       object
EverBenched                  object
ExperienceInCurrentDomain     int64
LeaveOrNot                    int64
dtype: object

## Opening CSV file in Spark

* ### Create `spark` object

In [5]:
spark = (
    SparkSession
        .builder
        .appName('PySpark Practice')
        .master('local[*]')
        .getOrCreate()
)
spark.sparkContext.setLogLevel("FATAL")

spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/25 19:51:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


* ### Read CSV

In [6]:
df = spark.read.format("csv").option("header", True).option("inferSchema", True).load('/kaggle/input/employee-dataset/Employee.csv')
df

DataFrame[Education: string, JoiningYear: int, City: string, PaymentTier: int, Age: int, Gender: string, EverBenched: string, ExperienceInCurrentDomain: int, LeaveOrNot: int]

In [7]:
df.printSchema()

root
 |-- Education: string (nullable = true)
 |-- JoiningYear: integer (nullable = true)
 |-- City: string (nullable = true)
 |-- PaymentTier: integer (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- EverBenched: string (nullable = true)
 |-- ExperienceInCurrentDomain: integer (nullable = true)
 |-- LeaveOrNot: integer (nullable = true)



* ### Data only shows up when an action is performed

In [8]:
df.show()

+---------+-----------+---------+-----------+---+------+-----------+-------------------------+----------+
|Education|JoiningYear|     City|PaymentTier|Age|Gender|EverBenched|ExperienceInCurrentDomain|LeaveOrNot|
+---------+-----------+---------+-----------+---+------+-----------+-------------------------+----------+
|Bachelors|       2017|Bangalore|          3| 34|  Male|         No|                        0|         0|
|Bachelors|       2013|     Pune|          1| 28|Female|         No|                        3|         1|
|Bachelors|       2014|New Delhi|          3| 38|Female|         No|                        2|         0|
|  Masters|       2016|Bangalore|          3| 27|  Male|         No|                        5|         1|
|  Masters|       2017|     Pune|          3| 24|  Male|        Yes|                        2|         1|
|Bachelors|       2016|Bangalore|          3| 22|  Male|         No|                        0|         0|
|Bachelors|       2015|New Delhi|          3| 

In [9]:
df.rdd.getNumPartitions()

1

* ### Filtering rows

In [10]:
df_1 = df.where("Age > 30")
df_1

DataFrame[Education: string, JoiningYear: int, City: string, PaymentTier: int, Age: int, Gender: string, EverBenched: string, ExperienceInCurrentDomain: int, LeaveOrNot: int]

* ### Filtering columns (selecting specific cols)

In [11]:
# ways to represent cols

df_1.City, df_1['Age'], col('Education'), expr('Gender')

(Column<'City'>, Column<'Age'>, Column<'Education'>, Column<'Gender'>)

In [12]:
df_1 = df_1.select(df_1.City, df_1['Age'])

- ### Data only shows up when collected

In [13]:
df_1.show()

+---------+---+
|     City|Age|
+---------+---+
|Bangalore| 34|
|New Delhi| 38|
|New Delhi| 38|
|Bangalore| 34|
|New Delhi| 37|
|     Pune| 34|
|     Pune| 32|
|Bangalore| 39|
|Bangalore| 37|
|Bangalore| 34|
|     Pune| 34|
|Bangalore| 37|
|New Delhi| 36|
|Bangalore| 37|
|     Pune| 31|
|Bangalore| 40|
|     Pune| 37|
|New Delhi| 34|
|Bangalore| 36|
|Bangalore| 36|
+---------+---+
only showing top 20 rows



* ### More streamlined selection using `selectExpr` + typecasting using `cast`

In [14]:
df_1 = df.selectExpr('Education as edu', 'cast(age as int) as age')

In [15]:
df_1.show()

+---------+---+
|      edu|age|
+---------+---+
|Bachelors| 34|
|Bachelors| 28|
|Bachelors| 38|
|  Masters| 27|
|  Masters| 24|
|Bachelors| 22|
|Bachelors| 38|
|Bachelors| 34|
|Bachelors| 23|
|  Masters| 37|
|  Masters| 27|
|Bachelors| 34|
|Bachelors| 32|
|Bachelors| 39|
|Bachelors| 37|
|Bachelors| 29|
|Bachelors| 34|
|Bachelors| 34|
|Bachelors| 30|
|Bachelors| 22|
+---------+---+
only showing top 20 rows



* ### Only selected columns show up in schema

In [16]:
df_1.printSchema()

root
 |-- edu: string (nullable = true)
 |-- age: integer (nullable = true)



* ### Adding columns

In [17]:
df_1 = df.withColumn('EverBenchedBool', df.EverBenched == 'Yes').withColumn('DidLeave', df.LeaveOrNot == 1)
df_1.show()

+---------+-----------+---------+-----------+---+------+-----------+-------------------------+----------+---------------+--------+
|Education|JoiningYear|     City|PaymentTier|Age|Gender|EverBenched|ExperienceInCurrentDomain|LeaveOrNot|EverBenchedBool|DidLeave|
+---------+-----------+---------+-----------+---+------+-----------+-------------------------+----------+---------------+--------+
|Bachelors|       2017|Bangalore|          3| 34|  Male|         No|                        0|         0|          false|   false|
|Bachelors|       2013|     Pune|          1| 28|Female|         No|                        3|         1|          false|    true|
|Bachelors|       2014|New Delhi|          3| 38|Female|         No|                        2|         0|          false|   false|
|  Masters|       2016|Bangalore|          3| 27|  Male|         No|                        5|         1|          false|    true|
|  Masters|       2017|     Pune|          3| 24|  Male|        Yes|               

In [18]:
columns = {
    'EverBenchedBool': df.EverBenched == 'Yes',
    'DidLeave': df.LeaveOrNot == 1
}

df_1 = df.withColumns(columns)
df_1.show()

+---------+-----------+---------+-----------+---+------+-----------+-------------------------+----------+---------------+--------+
|Education|JoiningYear|     City|PaymentTier|Age|Gender|EverBenched|ExperienceInCurrentDomain|LeaveOrNot|EverBenchedBool|DidLeave|
+---------+-----------+---------+-----------+---+------+-----------+-------------------------+----------+---------------+--------+
|Bachelors|       2017|Bangalore|          3| 34|  Male|         No|                        0|         0|          false|   false|
|Bachelors|       2013|     Pune|          1| 28|Female|         No|                        3|         1|          false|    true|
|Bachelors|       2014|New Delhi|          3| 38|Female|         No|                        2|         0|          false|   false|
|  Masters|       2016|Bangalore|          3| 27|  Male|         No|                        5|         1|          false|    true|
|  Masters|       2017|     Pune|          3| 24|  Male|        Yes|               

* ### Renaming columns

In [19]:
df_1 = df_1.withColumnRenamed('ExperienceInCurrentDomain', 'Exp')
df_1.show()

+---------+-----------+---------+-----------+---+------+-----------+---+----------+---------------+--------+
|Education|JoiningYear|     City|PaymentTier|Age|Gender|EverBenched|Exp|LeaveOrNot|EverBenchedBool|DidLeave|
+---------+-----------+---------+-----------+---+------+-----------+---+----------+---------------+--------+
|Bachelors|       2017|Bangalore|          3| 34|  Male|         No|  0|         0|          false|   false|
|Bachelors|       2013|     Pune|          1| 28|Female|         No|  3|         1|          false|    true|
|Bachelors|       2014|New Delhi|          3| 38|Female|         No|  2|         0|          false|   false|
|  Masters|       2016|Bangalore|          3| 27|  Male|         No|  5|         1|          false|    true|
|  Masters|       2017|     Pune|          3| 24|  Male|        Yes|  2|         1|           true|    true|
|Bachelors|       2016|Bangalore|          3| 22|  Male|         No|  0|         0|          false|   false|
|Bachelors|       2

* ### Dropping columns

In [20]:
df_1 = df_1.drop('EverBenched', 'LeaveOrNot') # not inplace, creates new object
df_1.show()

+---------+-----------+---------+-----------+---+------+---+---------------+--------+
|Education|JoiningYear|     City|PaymentTier|Age|Gender|Exp|EverBenchedBool|DidLeave|
+---------+-----------+---------+-----------+---+------+---+---------------+--------+
|Bachelors|       2017|Bangalore|          3| 34|  Male|  0|          false|   false|
|Bachelors|       2013|     Pune|          1| 28|Female|  3|          false|    true|
|Bachelors|       2014|New Delhi|          3| 38|Female|  2|          false|   false|
|  Masters|       2016|Bangalore|          3| 27|  Male|  5|          false|    true|
|  Masters|       2017|     Pune|          3| 24|  Male|  2|           true|    true|
|Bachelors|       2016|Bangalore|          3| 22|  Male|  0|          false|   false|
|Bachelors|       2015|New Delhi|          3| 38|  Male|  0|          false|   false|
|Bachelors|       2016|Bangalore|          3| 34|Female|  2|          false|    true|
|Bachelors|       2016|     Pune|          3| 23|  Mal

* ### Limit

In [21]:
df_lim = df_1.limit(5)
df_lim.show()

+---------+-----------+---------+-----------+---+------+---+---------------+--------+
|Education|JoiningYear|     City|PaymentTier|Age|Gender|Exp|EverBenchedBool|DidLeave|
+---------+-----------+---------+-----------+---+------+---+---------------+--------+
|Bachelors|       2017|Bangalore|          3| 34|  Male|  0|          false|   false|
|Bachelors|       2013|     Pune|          1| 28|Female|  3|          false|    true|
|Bachelors|       2014|New Delhi|          3| 38|Female|  2|          false|   false|
|  Masters|       2016|Bangalore|          3| 27|  Male|  5|          false|    true|
|  Masters|       2017|     Pune|          3| 24|  Male|  2|           true|    true|
+---------+-----------+---------+-----------+---+------+---+---------------+--------+



In [22]:
df_1.show(5)

+---------+-----------+---------+-----------+---+------+---+---------------+--------+
|Education|JoiningYear|     City|PaymentTier|Age|Gender|Exp|EverBenchedBool|DidLeave|
+---------+-----------+---------+-----------+---+------+---+---------------+--------+
|Bachelors|       2017|Bangalore|          3| 34|  Male|  0|          false|   false|
|Bachelors|       2013|     Pune|          1| 28|Female|  3|          false|    true|
|Bachelors|       2014|New Delhi|          3| 38|Female|  2|          false|   false|
|  Masters|       2016|Bangalore|          3| 27|  Male|  5|          false|    true|
|  Masters|       2017|     Pune|          3| 24|  Male|  2|           true|    true|
+---------+-----------+---------+-----------+---+------+---+---------------+--------+
only showing top 5 rows



* ### Writing to CSV

In [23]:
df_1.write.format('csv').save(f'/kaggle/working/df_{curr_time()}.csv')